# Elastic Stackの構築手順：収容（AWS）

----

Elasticsearchのインストール先となる Amazon&nbsp;EC2インスタンスを確保します。  


## 事前条件の確認

**事前に [01_01_Outline.ipynb#設定ファイルの出力](01_01_Outline.ipynb#設定ファイルの出力) を実行しておく必要があります。**


In [1]:
import os
assert os.path.exists('group_vars/all')


**上記がエラーになる場合は、[01_01_Outline.ipynb#設定ファイルの出力](01_01_Outline.ipynb#設定ファイルの出力)を実行してください。**

**事前にTerminalから、 `aws configure` を実施してください。**

`aws configure` の実施例:

```
$ aws configure
AWS Access Key ID [None]: (自身のアカウントのアクセスキー)
AWS Secret Access Key [None]: (自身のアカウントのシークレットアクセスキー)
Default region name [None]: ap-northeast-1 (使用したいリージョン)
Default output format [None]: json
```

正しくアクセス情報が指定されているかどうかを、`describe-account-attributes`により確認します。**以下のコマンド実行がエラーとなる場合、`aws configure`が正しくなされていない可能性があります。**

In [2]:
!aws ec2 describe-account-attributes

{
    "AccountAttributes": [
        {
            "AttributeName": "supported-platforms", 
            "AttributeValues": [
                {
                    "AttributeValue": "VPC"
                }
            ]
        }, 
        {
            "AttributeName": "vpc-max-security-groups-per-interface", 
            "AttributeValues": [
                {
                    "AttributeValue": "5"
                }
            ]
        }, 
        {
            "AttributeName": "max-elastic-ips", 
            "AttributeValues": [
                {
                    "AttributeValue": "5"
                }
            ]
        }, 
        {
            "AttributeName": "max-instances", 
            "AttributeValues": [
                {
                    "AttributeValue": "20"
                }
            ]
        }, 
        {
            "AttributeName": "vpc-max-elastic-ips", 
            "AttributeValues": [
                {
                    "AttributeValue": "5"
    

## 設定
インスタンスの生成に必要な情報を設定します。

マシンイメージを設定します。

In [3]:
#CentOS 7 (x86_64) - with Updates HVM
# ap-northeast-1 (アジアパシフィック東京リージョン) を使用する場合のAMI ID
image_id = 'ami-eec1c380'

# us-west-2 (オレゴン) を使用する場合
#image_id = 'ami-f4533694'

インスタンスタイプとインスタンス数を設定します。  
[Amazon EC2 インスタンス](https://aws.amazon.com/jp/ec2/instance-types/)のページから適切なタイプを選択してください。

例えば64GBのメモリを利用するのであれば **m4.4xlarge** が選択候補となります。  
利用したいサイズに応じてタイプを選んで下さい。

In [4]:
instance_type = 'm4.4xlarge'
instance_count = 1

データ蓄積ディスクとなる[EBS](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/AmazonEBS.html)の[ブロックデバイスマッピング](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/block-device-mapping-concepts.html)設定を行います。  
`VolumeSize`、`VolumeType`を設定します。  
`DeleteOnTermination`にtrueを設定すると、インスタンス破棄時にEBSが削除されます。  


In [5]:
block_device_mappings = {
    "DeviceName": "/dev/sda1",
    "Ebs": {
      "VolumeSize": 1024,
      "VolumeType": "gp2",
      "DeleteOnTermination": "true",
    }
}

### Keypairの準備

使用するキーペア名を設定します。

In [6]:
hostname = !hostname
keypair_name = 'es-jupyter-{}'.format(hostname[0])
keypair_name

'es-jupyter-6918f135xxxx'

In [7]:
import os
ssh_keypath = os.path.expanduser('~/.ssh')
if not os.path.exists(os.path.join(ssh_keypath, 'ansible_id_rsa')):
    !ssh-keygen -t rsa -b 2048 -C ansible -f {ssh_keypath}/ansible_id_rsa -P ''
!cat {ssh_keypath}/ansible_id_rsa.pub

ssh-rsa XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ansible


In [8]:
import json
result = !aws ec2 describe-key-pairs
keypairs = json.loads('\n'.join(result))['KeyPairs']
keypairs

[{u'KeyFingerprint': u'21:ca:56:ba:40:61:14:49:a5:f4:11:5c:xx:xx:xx:xx',
  u'KeyName': u'es-jupyter-6918f135xxxx'}]

既存キーペアを削除しておく。

In [9]:
if keypair_name in [k['KeyName'] for k in keypairs]:
    !aws ec2 delete-key-pair --key-name {keypair_name}

In [10]:
!aws ec2 import-key-pair --key-name {keypair_name} --public-key-material file://{ssh_keypath}/ansible_id_rsa.pub

{
    "KeyName": "es-jupyter-6918f135xxxx", 
    "KeyFingerprint": "21:ca:56:ba:40:61:14:49:a5:f4:11:5c:xx:xx:xx:xx"
}


### VPCの準備

Elasticsearch構築Notebookは、VPCに仮想マシンを確保します。

すでに作成してあるVPC, Subnetに仮想マシンを作成したい場合は、以下に入力してください。

In [11]:
# vpc_id = 'vpc-xxxxxxxx'
# subnet_id = 'subnet-xxxxxxxx'

`vpc_id` が指定されたなかった場合は、本Notebookは `es-jupyter` という名前のVPCを構築し、Internet Gatewayをアタッチします。
以下のセルを実行してください。

> VPC, Subnetが不要になったら、AWSコンソールからVPCに関連するリソースを削除してください。

In [12]:
if 'vpc_id' not in locals():
    vpc_cidr_block = '10.30.0.0/16'
    # VPCの検索
    result = !aws ec2 describe-vpcs
    vpcs = json.loads(''.join(result))['Vpcs']
    tagged_vpcs = [dict([(t['Key'], t['Value']) for t in vpc['Tags']] + [('VpcId', vpc['VpcId'])])
                   for vpc in vpcs if 'Tags' in vpc]
    target_vpcs = [v for v in tagged_vpcs if 'Name' in v and v['Name'] == 'es-jupyter']
    if len(target_vpcs) == 0:
        # VPCの作成
        result = !aws ec2 create-vpc --cidr-block {vpc_cidr_block}
        new_vpc_id = json.loads(''.join(result))['Vpc']['VpcId']
        print('VPC created: {}'.format(new_vpc_id))
        !aws ec2 create-tags --resources {new_vpc_id} --tags Key=Name,Value=es-jupyter

        # Subnetの作成
        result = !aws ec2 create-subnet --vpc-id {new_vpc_id} --cidr-block {vpc_cidr_block}
        new_subnet_id = json.loads(''.join(result))['Subnet']['SubnetId']
        print('Subnet created: {}'.format(new_subnet_id))
        !aws ec2 create-tags --resources {new_subnet_id} --tags Key=Name,Value=es-jupyter

        # Internet Gateway
        result = !aws ec2 create-internet-gateway
        new_internetgateway_id = json.loads(''.join(result))['InternetGateway']['InternetGatewayId']
        !aws ec2 attach-internet-gateway --internet-gateway-id {new_internetgateway_id} --vpc-id {new_vpc_id}
        !aws ec2 create-tags --resources {new_internetgateway_id} --tags Key=Name,Value=es-jupyter
        # route
        result = !aws ec2 describe-route-tables
        routes = json.loads(''.join(result))['RouteTables']
        target_routes = [r['RouteTableId']
                         for r in routes if 'VpcId' in r and r['VpcId'] == new_vpc_id]
        assert len(target_routes) > 0
        !aws ec2 create-route --route-table-id {target_routes[0]} --destination-cidr-block '0.0.0.0/0' --gateway-id {new_internetgateway_id}

        vpc_id = new_vpc_id
        subnet_id = new_subnet_id
    else:
        existing_vpc_id = target_vpcs[0]['VpcId']
        print('VPC found: {}'.format(existing_vpc_id))

        # Subnetの検索
        result = !aws ec2 describe-subnets
        subnets = json.loads(''.join(result))['Subnets']
        tagged_subnets = [dict([(t['Key'], t['Value']) for t in subnet['Tags']] + \
                               [('SubnetId', subnet['SubnetId']), ('VpcId', subnet['VpcId'])])
                          for subnet in subnets if 'Tags' in subnet and 'VpcId' in subnet]
        target_subnets = [v for v in tagged_subnets if 'Name' in v and v['Name'] == 'es-jupyter']
        target_subnets = [v for v in target_subnets if v['VpcId'] == existing_vpc_id]
        assert len(target_subnets) > 0
        existing_subnet_id = target_subnets[0]['SubnetId']
        print('Subnet found: {}'.format(existing_subnet_id))

        vpc_id = existing_vpc_id
        subnet_id = existing_subnet_id
(vpc_id, subnet_id)

VPC created: vpc-cd43xxxx
Subnet created: subnet-db8fxxxx
{
    "Return": true
}


(u'vpc-cd43xxxx', u'subnet-db8fxxxx')

### Security Groupの準備

Security GroupもVPC同様に準備します。

すでに作成してあるSecurity Groupを使用する場合は、以下にIDを設定してください。

In [13]:
#security_group_id = 'sg-xxxxxxxx'

VPC同様、IDが設定されていない場合は、Security Groupを生成します。自身のIPアドレス(範囲)を正しく設定してください。

In [14]:
if 'security_group_id' not in locals():
    # このJupyter Notebook Serverが動作している環境のグローバルIPアドレス範囲を設定
    # (ping, SSH可能な範囲が制限されます)
    # 
    # ホストを指定する場合
    # my_ips = ['xxx.xxx.xxx.xxx/32']
    # ネットワーク範囲を指定する場合
    # my_ips = ['xxx.xxx.xxx.xxx/yy']
    my_ips = 

    result = !aws ec2 describe-security-groups
    sgs = json.loads(''.join(result))['SecurityGroups']
    tagged_sgs = [dict([(t['Key'], t['Value']) for t in sg['Tags']] + \
                       [('GroupId', sg['GroupId']), ('VpcId', sg['VpcId'])])
                   for sg in sgs if 'Tags' in sg and 'VpcId' in sg]
    target_sgs = [sg for sg in tagged_sgs if 'Name' in sg and sg['Name'] == 'es-jupyter' and sg['VpcId'] == vpc_id]
    if len(target_sgs) == 0:
        result = !aws ec2 create-security-group --group-name es-jupyter --description es-jupyter --vpc-id {vpc_id}
        new_sg_id = json.loads(''.join(result))['GroupId']
        print('Security Group created: {}'.format(new_sg_id))
        !aws ec2 create-tags --resources {new_sg_id} --tags Key=Name,Value=es-jupyter
        for my_ip in my_ips:
            # SSH
            !aws ec2 authorize-security-group-ingress --group-id {new_sg_id} --cidr {my_ip} --protocol tcp --port 22
            # ICMP
            !aws ec2 authorize-security-group-ingress --group-id {new_sg_id} --cidr {my_ip} --protocol icmp --port -1
            # Elasticsearch
            !aws ec2 authorize-security-group-ingress --group-id {new_sg_id} --cidr {my_ip} --protocol tcp --port 9200
        !aws ec2 authorize-security-group-ingress --group-id {new_sg_id} --source-group {new_sg_id} --protocol all
        security_group_id = new_sg_id
    else:
        existing_sg_id = target_sgs[0]['GroupId']
        print('Security Group found: {}'.format(existing_sg_id))
        security_group_id = existing_sg_id
security_group_id

Security Group created: sg-d524xxxx


u'sg-d524xxxx'

## EC2インスタンスの起動・設定

サーバ構成に応じたEC2インスタンスを起動し、各種設定を実施します。


### インスタンスの起動

ここまでで設定した定義を用いてインスタンスを生成します。

以下では、AWSに対してインスタンス起動要求を発行しています。

> Elasticsearchの試用が終わったら、AWSコンソールからインスタンスを削除するようにしてください。

> **初めてこのNotebookを使ってインスタンスを起動する際**に、CentOSのAMIイメージの利用条件確認を求めるエラーメッセージが表示される場合があります。エラーメッセージ中にAMI MarketplaceのURLが表示されますので、利用条件を確認の上Acceptしてから再度実行してください。

In [15]:
%run scripts/get-json-repr.py

result = !aws ec2 run-instances --image-id {image_id} \
                                --count {instance_count} \
                                --instance-type {instance_type} \
                                --key-name {keypair_name} \
                                --block-device-mappings '{get_block_device_mappings_repr(block_device_mappings)}' \
                                --security-group-ids {security_group_id} \
                                --subnet-id {subnet_id}
try:
    instances = json.loads('\n'.join(result))['Instances']
except ValueError:
    raise ValueError('\n'.join(result))
instances

[{u'AmiLaunchIndex': 0,
  u'Architecture': u'x86_64',
  u'BlockDeviceMappings': [],
  u'ClientToken': u'',
  u'EbsOptimized': False,
  u'Hypervisor': u'xen',
  u'ImageId': u'ami-f4533694',
  u'InstanceId': u'i-0bf412a529d12xxxx',
  u'InstanceType': u'm4.4xlarge',
  u'KeyName': u'es-jupyter-6918f135xxxx',
  u'LaunchTime': u'2017-07-30T12:44:06.000Z',
  u'Monitoring': {u'State': u'disabled'},
  u'NetworkInterfaces': [{u'Attachment': {u'AttachTime': u'2017-07-30T12:44:06.000Z',
     u'AttachmentId': u'eni-attach-b36e825f',
     u'DeleteOnTermination': True,
     u'DeviceIndex': 0,
     u'Status': u'attaching'},
    u'Description': u'',
    u'Groups': [{u'GroupId': u'sg-d524xxxx', u'GroupName': u'es-jupyter'}],
    u'Ipv6Addresses': [],
    u'MacAddress': u'06:4b:3a:d7:81:6c',
    u'NetworkInterfaceId': u'eni-52ec176c',
    u'OwnerId': u'845836667502',
    u'PrivateIpAddress': u'10.30.xxx.xxx',
    u'PrivateIpAddresses': [{u'Primary': True,
      u'PrivateIpAddress': u'10.30.xxx.xxx'}],
  


生成したインスタンスIDを確認します。

In [16]:
id_list = map(lambda i: i['InstanceId'].encode('utf-8'), instances)
id_list_text = ' '.join(id_list)
id_list

['i-0bf412a529d12xxxx']


インスタンスの状態が running になるまで待ちます。

In [17]:
from functools import reduce
import time

retries = 10
while retries > 0:
    # インスタンス情報を取得
    desc_instances_stdout = !aws ec2 describe-instances --instance-ids $id_list_text
    try:
        json.loads('\n'.join(desc_instances_stdout))
    except:
        print('\n'.join(desc_instances_stdout))
    
    # JSON化し、インスタンスのリストとして取り出す
    desc_instances = json.loads('\n'.join(desc_instances_stdout))
    target_instances = reduce(lambda x, y: x + y, map(lambda r: r['Instances'], desc_instances['Reservations']))
    # IDと状態を出力
    status_text = '\n'.join(map(lambda i : '{0} : {1}'.format(i['InstanceId'], i['State']['Name']), target_instances))
    print(status_text)
    # runningになっていないものがまだあれば、スリープして最初に戻る。全てrunningなら終了。
    not_running_num = len(list(filter(lambda i : i['State']['Name'] != 'running', target_instances)))
    if not_running_num == 0:
        break
    print('Not running : {0}'.format(not_running_num))
    time.sleep(5)
    print('------')
    retries -= 1

for i in target_instances:
    assert i['State']['Name'] == 'running'
host_list = [i['PrivateIpAddress'] for i in target_instances]
host_list

i-0bf412a529d12xxxx : pending
Not running : 1
------
i-0bf412a529d12xxxx : running


[u'10.30.xxx.xxx']

### タグの設定

タグのNameキーに名称を設定します。

In [18]:
!aws ec2 create-tags --resources {id_list_text} --tags Key=Name,Value=es-jupyter

### Elastic IPの設定

インスタンスにElastic IPを設定します。

In [19]:
host_list = []
for i in target_instances:
    result = !aws ec2 allocate-address
    new_eip = json.loads(''.join(result))
    !aws ec2 associate-address --allocation-id {new_eip['AllocationId']} --instance-id {i['InstanceId']}
    host_list.append((i['InstanceId'], new_eip['PublicIp'], i['PrivateIpAddress']))
host_list

{
    "AssociationId": "eipassoc-2ba0xxxx"
}


[(u'i-0bf412a529d12xxxx', u'34.213.xx.xx', u'10.30.xxx.xxx')]

pingが通ることを確認します。

Instanceがrunningになった後、**<font color="red">pingが疎通するようになるまでしばらく(数分程度)</font>**かかる場合もあります。pingの実行が成功する(packet lossが0%)まで以下のセルの実行を試すようにしてください。

In [20]:
for iid, host, privip in host_list:
    !ping -c 4 {host}

PING 34.213.xx.xx (34.213.xx.xx): 56 data bytes
64 bytes from 34.213.xx.xx: icmp_seq=0 ttl=37 time=0.352 ms
64 bytes from 34.213.xx.xx: icmp_seq=1 ttl=37 time=0.595 ms
64 bytes from 34.213.xx.xx: icmp_seq=2 ttl=37 time=0.476 ms
64 bytes from 34.213.xx.xx: icmp_seq=3 ttl=37 time=0.295 ms
--- 34.213.xx.xx ping statistics ---
4 packets transmitted, 4 packets received, 0% packet loss
round-trip min/avg/max/stddev = 0.295/0.429/0.595/0.116 ms


全てのインスタンスに対しpingが通ればEC2インスタンス生成は完了です。

### Host Keyの確認

確保した仮想マシンのホストキーを確認します。Terminalからsshコマンドを実施し、Host Keyを確認してください。

(ここではログインに必要な情報は設定していませんので、Host Keyを承認するのみで、Permission deniedとなります。)

まず、sshコマンド列を生成します。

In [21]:
for iid, host, privip in host_list:
    print('---- SSH: {}'.format(iid))
    print('ssh {}\n'.format(host))

---- SSH: i-0bf412a529d12xxxx
ssh 34.213.xx.xx



EC2から、仮想マシンに設定されたHost Keyを確認します。

起動後しばらくは **<font color="red">Outputの内容が得られずエラー(KeyError: 'Output')となる場合があります。</font>** この状態が長く続く場合、AWSコンソールから [システムログの取得] で確認してみてください。

In [29]:
for iid, host, privip in host_list:
    print('---- BEGIN: {}'.format(iid))
    result = !aws ec2 get-console-output --instance-id {iid}
    try:
        print('\n'.join([line
                         for line in json.loads(''.join(result))['Output'].split('\n')
                         if line.startswith('ec2: ')]))
    except KeyError:
        print('No output')
    print('---- END: {}\n\n'.format(iid))

---- BEGIN: i-0bf412a529d12xxxx
ec2: 
ec2: #############################################################
ec2: -----BEGIN SSH HOST KEY FINGERPRINTS-----
ec2: 256 7d:9a:ef:65:0d:47:58:7a:17:ab:ef:43:9d:ec:bb:d8   (ECDSA)
ec2: 256 39:ed:31:a6:c0:4a:97:9b:22:92:98:31:e9:6b:ee:81   (ED25519)
ec2: 2048 82:1f:a0:64:63:42:60:65:6d:47:6f:98:0e:be:5a:94   (RSA)
ec2: -----END SSH HOST KEY FINGERPRINTS-----
ec2: #############################################################
---- END: i-0bf412a529d12xxxx




上記のコマンドをJupyterのTerminalから実行すると、Host keyのFingerprintが表示されます。ここで表示されたFingerprintが、以下のセルの実行結果にあるSSH HOST KEY FINGERPRINTSと合致することを確認した上で、 *yes* を入力してください。

SSHコマンドの出力例:

```
$ ssh 52.26.xxx.xxx
The authenticity of host '52.26.xxx.xxx (52.26.xxx.xxx)' can't be established.
ECDSA key fingerprint is 2e:da:4e:3c:a7:a3:79:bc:e0:93:35:4b:xx:xx:xx:xx.
Are you sure you want to continue connecting (yes/no)?
```

## セキュリティグループへのホスト情報の登録

各インスタンスのPrivate IPアドレスを、セキュリティグループの許可リストに追加します。

インスタンスごとに、9200番ポート（外部通信用）と9300番ポート（クラスタ内部通信用）の両方を許可します。

In [30]:
for ip in [private_ip for iid, pubip, private_ip in host_list]:
    !aws ec2 authorize-security-group-ingress --group-id {security_group_id} --protocol tcp --port 9200 --cidr {ip}/32
    !aws ec2 authorize-security-group-ingress --group-id {security_group_id} --protocol tcp --port 9300 --cidr {ip}/32

## Inventory作成用のインスタンスリストを作成
ホストのパブリックDNS一覧をリスト化します  

次のブックの[Inventory作成](01_03_Set_Inventory.ipynb)で使用します。  
出現した文字列をクリップボードにコピーしてください。

In [31]:
print('host_list = {}'.format(repr(dict([('host{}'.format(i + 1), h) for i, h in enumerate(host_list)]))))

host_list = {'host1': (u'i-0bf412a529d12xxxx', u'34.213.xx.xx', u'10.30.xxx.xxx')}
